In [ ]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 41.29 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [ ]:
import csv
data = {}
with open("/content/facial_expressions/data/legend.csv") as f:
  reader = csv.reader(f)
  next(reader)
  for r in reader:
    key = r[2].lower()
    if key in data:
      data[key].append(r[1])
    else:
      data[key] = [r[1]]


In [ ]:
list_emotions = list(data.keys())
list_emotions

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [ ]:
import os
os.mkdir("master")
os.mkdir("master/train_data")
os.mkdir("master/test_data")

In [ ]:
for emotion in list_emotions:
  os.mkdir(os.path.join("master/train_data/" , emotion))
  os.mkdir(os.path.join("master/test_data/" , emotion))

In [ ]:
from shutil import copyfile

t_t_split = 0.8

#data has image name, emotion

for emotion, image_list in data.items():
  split_length = int(t_t_split * len(image_list))
  train_images = image_list[:split_length]
  test_images = image_list[split_length:]
  
  for image in train_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master/train_data', emotion , image)
    copyfile(source, dest)
  for image in test_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master/test_data', emotion , image)
    copyfile(source, dest)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
train_dir = '/content/master/train_data'
test_dir = '/content/master/test_data'

train_datagen = ImageDataGenerator(rescale=1.0/ 255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (100,100),
    class_mode = 'categorical',
    batch_size = 128
)
test_datagen = ImageDataGenerator(rescale=1.0/ 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (100,100),
    class_mode = 'categorical',
    batch_size = 128
)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense,Flatten

In [ ]:
model = tf.keras.models.Sequential([
                               Conv2D(16, (3, 3), activation = 'relu', input_shape = (100,100,3)),
                               MaxPooling2D(2,2),
                               Conv2D(32, (3, 3), activation = 'relu'),
                               MaxPooling2D(2,2),
                               Conv2D(64, (3, 3), activation = 'relu'),
                               MaxPooling2D(2,2),
                               Flatten(),
                               Dense(1024, activation = 'relu'),
                               Dense(8, activation = 'softmax')

]
)

model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 49, 49, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_27 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 10, 10, 64)      

In [ ]:
es = EarlyStopping(monitor = 'val_acc', patience = 2, min_delta = 0.01)

In [ ]:
model.fit_generator(
    train_generator,
    epochs = 10,
    verbose = 1,
    validation_data = test_generator,
    callbacks = [es]
)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


86/86 [==============================] - 16s 186ms/step - loss: 1.4365 - acc: 0.4784 - val_loss: 1.0574 - val_acc: 0.5011
Epoch 2/10
86/86 [==============================] - 15s 179ms/step - loss: 1.0475 - acc: 0.4838 - val_loss: 1.0428 - val_acc: 0.5011
Epoch 3/10
86/86 [==============================] - 15s 180ms/step - loss: 1.0411 - acc: 0.4926 - val_loss: 1.0439 - val_acc: 0.5011
